In [1]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

# For data preprocess
import numpy as np
import pandas as pd
from pandas import DataFrame


# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.rcParams['font.sans-serif']=['SimHei'] #pyplot中文显示



In [ ]:
use_gpu=torch.cuda.is_available()

In [2]:
'''
#data = [['Google',10],['Runoob',12],['Wiki',13]]
global df
df = pd.DataFrame(columns=["风场","风机","时段","时刻","风速","风向"])
df.to_csv('C:/Users/62331/Desktop/ML/机组数据驱动的风电场短期风况预测/ans.csv',sep=',',index=False,header=True) 
'''
#测试集机组2的40冬，44秋不全

'\n#data = [[\'Google\',10],[\'Runoob\',12],[\'Wiki\',13]]\nglobal df\ndf = pd.DataFrame(columns=["风场","风机","时段","时刻","风速","风向"])\ndf.to_csv(\'C:/Users/62331/Desktop/ML/机组数据驱动的风电场短期风况预测/ans.csv\',sep=\',\',index=False,header=True) \n'

In [2]:
class RNN (nn.Module):
    def __init__(self):
        super(RNN,self).__init__() 
        self.lstm = nn.LSTM(20,8,2) 
        self.out = nn.Linear(8,20) 
        self.dropout = nn.Dropout(p=0.3)
    def forward(self,x):
        x1,_ = self.lstm(x)
        a,b,c = x1.shape
        out = self.out(x1.view(-1,c))#将lstm输出的三维数据x1调整成二维数据，最后的特征维度不能变
        out = self.dropout(out)
        out1 = out.view(a,b,-1) #把二维的out调整成三维数据，下一次循环使用
        return out1
rnn = RNN()

#参数寻优，计算损失函数
optimizer = torch.optim.Adam(rnn.parameters(),lr = 0.02)
loss_func = nn.MSELoss()

In [3]:
def data_processing(train_data,ans_data0,keyword):
    time_stamp=20 #十分钟的数据，题目本来要求是利用一小时的数据，但如果用了1h(time_stamp=120),训练效果emmmmmm
    train = pd.read_csv(train_data)  #训练集
    #'C:/Users/62331/Desktop/ML/机组数据驱动的风电场短期风况预测/风场1/x26/2018-07-30.csv'
    train=train.dropna()
    #train=train[['变频器电网侧有功功率', '外界温度', '风向', '风速']]
    train=train[[keyword]]
    x_train= []
    y_train= []
    for i in range(time_stamp, len(train)-21):
        x_train.append(train[i-time_stamp:i])
        y_train.append(train[i:i+20])
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_train = x_train.reshape(-1, 1, 20) #将训练数据调整成pytorch中lstm算法的输入维度
    y_train = y_train.reshape(-1, 1, 20)  #将目标值调整成pytorch中lstm算法的输出维度
    x_train = torch.from_numpy(x_train)
    y_train = torch.from_numpy(y_train)
    print("read finished,start train")

    for i in range(100):
        var_x = Variable(x_train).type(torch.FloatTensor)
        var_y = Variable(y_train).type(torch.FloatTensor)
        out = rnn(var_x)
        loss = loss_func(out,var_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #if (i+1)%100==0:
        #    print('Epoch:{}, Loss:{:.5f}'.format(i+1, loss.item()))
            
    
    
    
    
    
    print("train finished")
    #预测        
    for i in range(1,10):
        ans_data=ans_data0+"_0"+str(i)+".csv"
        ans = pd.read_csv(ans_data) 
        ans = ans[[keyword]]
        x_ans= []
        x_ans.append(ans[-time_stamp-1:-1])
        x_ans = np.array(x_ans)
        x_ans = x_ans.reshape(-1, 1, 20) #将训练数据调整成pytorch中lstm算法的输入维度
        x_ans = torch.from_numpy(x_ans)
        x_ans = Variable(x_ans).type(torch.FloatTensor)
        pred = rnn(x_ans)
        pred_test = pred.view(-1).data.numpy()  #转换成一维的ndarray数据，这是预测值
        
        outpath=ans_data[:15]+"ans/"+keyword+"/"+ans_data[22:26]+ans_data[26:29]+ans_data[-8:]
        df = pd.DataFrame(pred_test,columns=[keyword])
        df.to_csv(outpath,index=False,header=True)
        
    print("Finished writing to ",outpath)
        
        
        
        
    for i in range(10,21):
        ans_data=ans_data0+"_"+str(i)+".csv"
        ans = pd.read_csv(ans_data) 
        ans = ans[[keyword]]
        x_ans= []
        x_ans.append(ans[-time_stamp-1:-1])
        x_ans = np.array(x_ans)
        x_ans = x_ans.reshape(-1, 1, 20) #将训练数据调整成pytorch中lstm算法的输入维度
        x_ans = torch.from_numpy(x_ans)
        x_ans = Variable(x_ans).type(torch.FloatTensor)
        pred = rnn(x_ans)
        pred_test = pred.view(-1).data.numpy()  #转换成一维的ndarray数据，这是预测值
        
        outpath=ans_data[:15]+"ans/"+keyword+"/"+ans_data[22:26]+ans_data[26:29]+ans_data[-8:]
        df = pd.DataFrame(pred_test,columns=[keyword])
        df.to_csv(outpath,index=False,header=True)
        
    print("Finished writing to ",outpath)
        


    
    
#data_processing('C:/Users/62331/Desktop/ML/机组数据驱动的风电场短期风况预测/风场1/x26/2018-07-30.csv','C:/Users/62331/Desktop/ML/机组数据驱动的风电场短期风况预测/风场1/x27/2018-07-31.csv','风速')


In [4]:
location='风场1'
Path_data='D:/ML/HFenergy/风场1/x'
Path_test='D:/ML/HFenergy/测试集_初赛/风场1/x'
i=29
test=Path_test+str(i)+'/夏'
ans_data=test
i=15
ans_data=ans_data+"_"+str(i)+".csv"
print(ans_data)
keyword="风速"

outpath=ans_data[:15]+"ans/"+keyword+"/"+ans_data[22:26]+ans_data[26:29]+ans_data[-8:]
print(outpath)


D:/ML/HFenergy/测试集_初赛/风场1/x29/夏_15.csv
D:/ML/HFenergy/ans/风速/风场1/x29夏_15.csv


In [5]:
#批量乱搞
location='风场1'
Path_data='D:/ML/HFenergy/风场1/x'
Path_test='D:/ML/HFenergy/测试集_初赛/风场1/x'
for i in range (26,51):
    train=Path_data+str(i)+'/2018-02-01.csv'
    test=Path_test+str(i)+'/春'
    data_processing(train,test,'风向')
    data_processing(train,test,'风速')



read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x26春_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x26春_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x26春_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x26春_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x27春_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x27春_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x27春_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x27春_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x28春_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x28春_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x28春_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x28春_20.csv
read finished,start train
train finished
Finis

In [7]:
Path_data='D:/ML/HFenergy/风场1/x'
Path_test='D:/ML/HFenergy/测试集_初赛/风场1/x'
for i in range (26,51):
    train=Path_data+str(i)+'/2018-05-01.csv'
    test=Path_test+str(i)+'/夏'
    data_processing(train,test,'风向')
    data_processing(train,test,'风速')
    


read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x26夏_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x26夏_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x26夏_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x26夏_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x27夏_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x27夏_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x27夏_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x27夏_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x28夏_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x28夏_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x28夏_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x28夏_20.csv
read finished,start train
train finished
Finis

In [ ]:
Path_data='C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/风场1/x'
Path_test='C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/测试集_初赛/风场1/x'
for i in range (26,51):
    train=Path_data+str(i)+'/2018-08-01.csv'
    test=Path_test+str(i)+'/秋'
    data_processing(train,test,'风向')
    data_processing(train,test,'风速')



read finished,start train
train finished
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风向/风场1/x26秋_09.csv
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风向/风场1/x26秋_20.csv
read finished,start train
train finished
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风速/风场1/x26秋_09.csv
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风速/风场1/x26秋_20.csv
read finished,start train
train finished
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风向/风场1/x27秋_09.csv
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风向/风场1/x27秋_20.csv
read finished,start train
train finished
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风速/风场1/x27秋_09.csv
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风速/风场1/x27秋_20.csv
read finished,start train


In [9]:
Path_data='D:/ML/HFenergy/风场1/x'
Path_test='D:/ML/HFenergy/测试集_初赛/风场1/x'
for i in range (26,51):
    train=Path_data+str(i)+'/2018-11-01.csv'
    test=Path_test+str(i)+'/冬'
    data_processing(train,test,'风向')
    data_proc essing(train,test,'风速') 
    



read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x26冬_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x26冬_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x26冬_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x26冬_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x27冬_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x27冬_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x27冬_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x27冬_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x28冬_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x28冬_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x28冬_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x28冬_20.csv
read finished,start train
train finished
Finis

In [6]:
location='风场2'
Path_data='C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/风场2/x'
Path_test='C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/测试集_初赛/风场2/x'
for i in range (25,50):
    train=Path_data+str(i)+'/2017-02-01.csv'
    test=Path_test+str(i)+'/春'
    data_processing(train,test,'风向')
    data_processing(train,test,'风速')



read finished,start train
train finished
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风向/风场2/x25春_09.csv
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风向/风场2/x25春_20.csv
read finished,start train
train finished
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风速/风场2/x25春_09.csv
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风速/风场2/x25春_20.csv
read finished,start train
train finished
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风向/风场2/x26春_09.csv
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风向/风场2/x26春_20.csv
read finished,start train
train finished
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风速/风场2/x26春_09.csv
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风速/风场2/x26春_20.csv
read finished,start train
train finished
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风向/风场2/x27春_09.csv
Finished writing to  

read finished,start train
train finished
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风速/风场2/x44春_09.csv
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风速/风场2/x44春_20.csv
read finished,start train
train finished
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风向/风场2/x45春_09.csv
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风向/风场2/x45春_20.csv
read finished,start train
train finished
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风速/风场2/x45春_09.csv
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风速/风场2/x45春_20.csv
read finished,start train
train finished
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风向/风场2/x46春_09.csv
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风向/风场2/x46春_20.csv
read finished,start train
train finished
Finished writing to  C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/ans/风速/风场2/x46春_09.csv
Finished writing to  

In [11]:
Path_data='D:/ML/HFenergy/风场2/x'
Path_test='D:/ML/HFenergy/测试集_初赛/风场2/x'
for i in range (25,50):
    train=Path_data+str(i)+'/2017-05-01.csv'
    test=Path_test+str(i)+'/夏'
    data_processing(train,test,'风向')
    data_processing(train,test,'风速')



read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场2/x25夏_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场2/x25夏_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x25夏_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x25夏_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场2/x26夏_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场2/x26夏_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x26夏_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x26夏_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场2/x27夏_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场2/x27夏_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x27夏_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x27夏_20.csv
read finished,start train
train finished
Finis

In [16]:
Path_data='C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/风场2/x'
Path_test='C:/Users/User/Desktop/ML/机组数据驱动的风电场短期风况预测/测试集_初赛/风场2/x'
for i in range (25,50):
    train=Path_data+str(i)+'/2018-07-01.csv'
    test=Path_test+str(i)+'/秋'
    data_processing(train,test,'风向')
    
    
    data_processing(train,test,'风速')



SyntaxError: invalid syntax (Temp/ipykernel_16340/2229674068.py, line 6)

In [20]:
Path_data='D:/ML/HFenergy/风场2/x'
Path_test='D:/ML/HFenergy/测试集_初赛/风场2/x'
for i in range (45,50):
    train=Path_data+str(i)+'/2017-11-01.csv'
    test=Path_test+str(i)+'/冬'
    #data_processing(train,test,'风向')
    data_processing(train,test,'风速')

read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x45冬_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x45冬_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x46冬_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x46冬_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x47冬_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x47冬_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x48冬_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x48冬_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x49冬_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x49冬_20.csv


In [25]:
#整合ans
import os

ans = pd.DataFrame(None,columns=["风场","风机","时段","时刻","风速","风向"])

#风场1

season=["春_","夏_","秋_","冬_"]
for i in range(26,51):
    machine='x'+str(i)
    for j in range(0,4):
        for k in range(1,10):
            cur_season=season[j]
            cur_season+='0'
            cur_season+=str(k)
            dir_path='D:/ML/HFenergy/ans/风向/风场1/'
            dir_path=dir_path+machine+cur_season+".csv"
            cur_dir=pd.read_csv(dir_path)
            
            spd_path='D:/ML/HFenergy/ans/风速/风场1/'
            spd_path=spd_path+machine+cur_season+".csv"
            cur_spd=pd.read_csv(spd_path)
            for l in range(0,20):
                cur_data=pd.DataFrame([["风场1",machine,cur_season,(l+1)*30,cur_spd["风速"][l],cur_dir["风向"][l]]],columns=["风场","风机","时段","时刻","风速","风向"])
                ans=ans.append(cur_data)
        for k in range(11,21):
            cur_season=season[j]
            cur_season+=str(k)
            dir_path='D:/ML/HFenergy/ans/风向/风场1/'
            dir_path=dir_path+machine+cur_season+".csv"
            cur_dir=pd.read_csv(dir_path)
            
            spd_path='D:/ML/HFenergy/ans/风速/风场1/'
            spd_path=spd_path+machine+cur_season+".csv"
            cur_spd=pd.read_csv(spd_path)
            for l in range(0,20):
                cur_data=pd.DataFrame([["风场1",machine,cur_season,(l+1)*30,cur_spd["风速"][l],cur_dir["风向"][l]]],columns=["风场","风机","时段","时刻","风速","风向"])
                ans=ans.append(cur_data)
#风场2
season=["春_","夏_","秋_","冬_"]
for i in range(25,50):
    machine='x'+str(i)
    for j in range(0,4):
        for k in range(1,10):
            cur_season=season[j]
            cur_season+='0'
            cur_season+=str(k)
            dir_path='D:/ML/HFenergy/ans/风向/风场2/'
            dir_path=dir_path+machine+cur_season+".csv"
            if(os.path.exists(dir_path)==False):
                continue
            cur_dir=pd.read_csv(dir_path)
            
            spd_path='D:/ML/HFenergy/ans/风速/风场2/'
            spd_path=spd_path+machine+cur_season+".csv"
            cur_spd=pd.read_csv(spd_path)
            for l in range(0,20):
                cur_data=pd.DataFrame([["风场2",machine,cur_season,(l+1)*30,cur_spd["风速"][l],cur_dir["风向"][l]]],columns=["风场","风机","时段","时刻","风速","风向"])
                ans=ans.append(cur_data)
        for k in range(11,21):
            cur_season=season[j]
            cur_season+=str(k)
            dir_path='D:/ML/HFenergy/ans/风向/风场2/'
            dir_path=dir_path+machine+cur_season+".csv"
            if(os.path.exists(dir_path)==False):
                continue
            cur_dir=pd.read_csv(dir_path)
            spd_path='D:/ML/HFenergy/ans/风速/风场2/'
            spd_path=spd_path+machine+cur_season+".csv"
            cur_spd=pd.read_csv(spd_path)
            for l in range(0,20):
                cur_data=pd.DataFrame([["风场2",machine,cur_season,(l+1)*30,cur_spd["风速"][l],cur_dir["风向"][l]]],columns=["风场","风机","时段","时刻","风速","风向"])
                ans=ans.append(cur_data)
ans.to_csv('D:/ML/HFenergy/pred.csv',index=False,header=True)